# Lab 04: AIP with Strands Agents - Multi-Tenant Marketing Platform

## Business Context
You are building a **Multi-Tenant Marketing Platform** using **Strands Agents** that serves multiple enterprise clients. Your platform uses AI agents to generate sophisticated marketing campaigns for different tenants:

- **Tenant A**: B2B tech company needing product launch campaigns
- **Tenant B**: B2C retail company needing promotional campaigns

**Challenge**: Track agent usage, costs, and performance per client using Application Inference Profiles for accurate billing and SLA monitoring.

## Learning Objectives
- Integrate Application Inference Profiles with Strands Agents framework
- Create tenant-specific agents with AIP-enabled BedrockModel
- Define custom tools using the `@tool` decorator
- Leverage automatic agent loop and tool orchestration
- Access built-in agent metrics and observability
- Compare Boto3 vs Strands approaches for multi-tenant AI

## What's Different from Lab 03?
- **Lab 03 (Boto3)**: Direct API calls, manual tool logic, manual metrics tracking
- **Lab 04 (Strands)**: Agent framework, automatic tool orchestration, built-in observability

In [ ]:
# Install required packages
!pip install --force-reinstall -r requirements.txt --quiet

## Section 1: Setup Strands Agents and Application Inference Profiles

First, let's set up our Strands agents framework and check for existing Application Inference Profiles from Lab 03.

In [ ]:
import boto3
import json
import time
from datetime import datetime
from typing import Dict, Any, List

# Import Strands framework
from strands import Agent, tool
from strands.models import BedrockModel

# Import lab helpers
from lab_helpers.config import Region, ModelId
from lab_helpers.aip_manager import AIPManager
from lab_helpers.usage_tracker import UsageTracker

# Initialize AWS clients (still needed for AIP management)
bedrock_client = boto3.client('bedrock', region_name=Region)
sts = boto3.client('sts', region_name=Region)

print(f"✅ Initialized Strands and boto3 clients for region: {Region}")
print(f"📋 Using Claude Sonnet 4 model: {ModelId}")

### 🏗️ Setup: Tenant Configurations

Define configurations for two tenants (same as Lab 03):
- **Tenant A (B2B Tech)**: SaaS DevOps platform marketing
- **Tenant B (B2C Retail)**: Fashion e-commerce marketing

Each tenant will get a Strands agent powered by their Application Inference Profile.

In [ ]:
# Initialize AIP Manager
aip_manager = AIPManager(bedrock_client)

# Define tenant configurations (same as Lab 03)
TENANT_CONFIGS = {
    "tenant_a": {
        "name": "marketing-ai-tenant-a",
        "description": "Marketing AI AIP for Tenant A",
        "tags": {
            "TenantId": "tenant-a",
            "BusinessType": "B2B-Tech",
            "Environment": "production",
            "CostCenter": "marketing-ai-platform"
        }
    },
    "tenant_b": {
        "name": "marketing-ai-tenant-b", 
        "description": "Marketing AI AIP for Tenant B",
        "tags": {
            "TenantId": "tenant-b",
            "BusinessType": "B2C-Retail",
            "Environment": "production",
            "CostCenter": "marketing-ai-platform"
        }
    }
}

print("📋 Tenant configurations defined:")
for tenant_id, config in TENANT_CONFIGS.items():
    print(f"  - {tenant_id}: {config['description']}")

### 🔍 Check for Existing AIPs (Reuse from Lab 03)

Before creating new Application Inference Profiles, let's check if they already exist from Lab 03.

**Strategy:**
1. Check if AIPs exist with the same names
2. If exists: **Reuse them** (saves time and avoids duplicates)
3. If not exists: **Create new ones** using the same logic as Lab 03

This approach ensures we can run Lab 04 independently or after Lab 03.

In [ ]:
# Check for existing AIPs and reuse or create
tenant_aips = {}

for tenant_id, config in TENANT_CONFIGS.items():
    print(f"\n🔍 Checking AIP for {tenant_id}...")
    
    try:
        # Check if AIP already exists (possibly from Lab 03)
        existing_arn = aip_manager.check_aip_exists(config["name"])
        
        if existing_arn:
            print(f"✅ Found existing AIP (reusing from Lab 03 or previous run)")
            print(f"   ARN: {existing_arn}")
            tenant_aips[tenant_id] = existing_arn
        else:
            print(f"📝 AIP not found - creating new one...")
            
            # Prepare tags
            tag_list = []
            if config["tags"]:
                tag_list = [{"key": k, "value": v} for k, v in config["tags"].items()]
            
            # Get account ID
            account_id = sts.get_caller_identity()['Account']
            
            # Construct proper ARN
            MODEL_ARN = f"arn:aws:bedrock:{Region}:{account_id}:inference-profile/{ModelId}"
            print(f"   Model ARN: {MODEL_ARN}")
            
            # Create Application Inference Profile
            response = bedrock_client.create_inference_profile(
                inferenceProfileName=config["name"],
                description=config["description"],
                modelSource={"copyFrom": MODEL_ARN},
                tags=tag_list
            )
            
            aip_arn = response['inferenceProfileArn']
            tenant_aips[tenant_id] = aip_arn
            
            print(f"✅ Created new AIP for {tenant_id}")
            print(f"   Status: {response['status']}")
            print(f"   ARN: {aip_arn}")
            
    except Exception as e:
        print(f"❌ Error with AIP for {tenant_id}: {str(e)}")

print(f"\n📊 Summary: {len(tenant_aips)} Application Inference Profiles ready for Strands agents")

### 🔍 System vs Application Inference Profiles

Quick reminder of what we've set up:

In [ ]:
# Compare System vs Application Inference Profile usage
print("🔍 SYSTEM vs APPLICATION INFERENCE PROFILES")
print("=" * 55)

print("\n📋 SYSTEM INFERENCE PROFILE (Traditional):")
print(f"   Model ID: {ModelId}")
print("   Strands Usage: Agent(model='us.anthropic.claude-sonnet-4...')")
print("   Tracking: ❌ No tenant-level visibility")
print("   Cost Allocation: ❌ Shared across all tenants")

print("\n📋 APPLICATION INFERENCE PROFILES (Multi-tenant):")
for tenant_id, aip_arn in tenant_aips.items():
    print(f"\n   🏢 {tenant_id.upper()}:")
    print(f"      AIP ARN: {aip_arn}")
    print(f"      Strands Usage: BedrockModel(model_id='{aip_arn}')")
    print("      Tracking: ✅ Per-tenant agent metrics")
    print("      Cost Allocation: ✅ Isolated per tenant")

print("\n💡 KEY INSIGHT: Strands agents + AIPs = Automatic tenant-aware observability!")

### 📊 CloudWatch Monitoring Function

Same monitoring function as Lab 03 - works with both boto3 and Strands approaches.

In [ ]:
def monitor_aip_usage(tenant_aips, region):
    """
    Monitor CloudWatch metrics for Application Inference Profiles.
    Works with both boto3 (Lab 03) and Strands (Lab 04) approaches.
    """
    from lab_helpers.cloudwatch import fetch_metrices, plot_graph

    print("📊 Fetching CloudWatch metrics for Application Inference Profiles...")
    print(f"Region: {region}")
    print(f"Time Range: Last 60 minutes")
    print("="*80)

    tenant_metrics = {}

    for tenant_id, aip_arn in tenant_aips.items():
        print(f"\n🏢 TENANT: {tenant_id.upper()}")
        print("="*50)
        print(f"AIP ARN: {aip_arn}")
        
        # Extract AIP ID for CloudWatch
        aip_id = aip_arn.split('/')[-1]
        print(f"AIP ID: {aip_id}")
        
        try:
            print(f"\n📊 METRICS FOR {tenant_id.upper()}:")
            response, input_token_response, output_token_response = fetch_metrices(
                Region=region,
                Period=60,
                Timedelta=60,
                Id=aip_id
            )
            
            tenant_metrics[tenant_id] = {
                'invocations': response,
                'input_tokens': input_token_response,
                'output_tokens': output_token_response
            }
            
            print(f"\n📈 USAGE PLOTS FOR {tenant_id.upper()}:")
            plot_graph(response, input_token_response, output_token_response)
            
            print("="*50)
                
        except Exception as e:
            print(f"⚠️ CloudWatch error for {tenant_id}: {str(e)}")
            print(f"💡 Metrics may take a few minutes to appear after agent invocations")
            print("="*50)
    
    return tenant_metrics

print("✅ CloudWatch monitoring function defined")

### 🔵 BEFORE Check: Baseline CloudWatch Metrics

Check CloudWatch metrics before running any Strands agents to establish a baseline.

In [ ]:
# Check CloudWatch metrics BEFORE any agent invocations
print("🔵 BEFORE CHECK: Querying CloudWatch for baseline metrics...")
print("="*80)
print("Expected: Empty if Lab 03 has not been run")
print("="*80 + "\n")

baseline_metrics = monitor_aip_usage(tenant_aips, Region)

print("\n" + "="*80)
print("✅ Baseline check complete!")
print("💡 Note: Empty metrics are expected if this is your first run")
print("="*80)

## Section 2: Define Marketing Tools and Create Strands Agents

Now let's define custom tools for our marketing agents using the `@tool` decorator and create tenant-specific agents.

### 🛠️ Define Marketing Tools with @tool Decorator

Strands agents can use custom tools defined as Python functions with the `@tool` decorator.

**Key features:**
- Automatic tool discovery by the agent
- Type hints become tool schema
- Docstrings provide tool descriptions to the LLM
- Automatic parameter validation

In [ ]:
# Define custom marketing tools using @tool decorator

@tool
def analyze_target_audience(
    business_type: str,
    product_category: str,
    age_range: str = "25-45"
) -> str:
    """
    Analyze target audience demographics and psychographics for marketing campaigns.
    
    Args:
        business_type: Type of business (B2B or B2C)
        product_category: Product or service category (e.g., 'SaaS', 'Fashion', 'Technology')
        age_range: Target age range (default: '25-45')
    
    Returns:
        Detailed target audience analysis with demographics and psychographics
    """
    
    # Simulate audience analysis
    analysis = f"""
Target Audience Analysis:
- Business Type: {business_type}
- Product Category: {product_category}
- Age Range: {age_range}

Demographics:
- Primary decision makers in the {age_range} age bracket
- {'Tech-savvy professionals with purchasing authority' if business_type == 'B2B' else 'Consumers with disposable income'}

Psychographics:
- Values: {'Efficiency, ROI, innovation' if business_type == 'B2B' else 'Quality, style, sustainability'}
- Pain Points: {'Complex workflows, high operational costs' if business_type == 'B2B' else 'Limited choices, poor quality products'}
- Goals: {'Streamline operations, reduce costs' if business_type == 'B2B' else 'Find quality products, express personal style'}
"""
    
    return analysis


@tool
def recommend_marketing_channels(
    business_type: str,
    budget: int,
    campaign_duration: int
) -> str:
    """
    Recommend optimal marketing channels based on business type, budget, and campaign duration.
    
    Args:
        business_type: Type of business (B2B or B2C)
        budget: Campaign budget in USD
        campaign_duration: Campaign duration in months
    
    Returns:
        Recommended marketing channels with budget allocation
    """
    
    if business_type == "B2B":
        channels = f"""
Recommended Marketing Channels (B2B):
Budget: ${budget:,} over {campaign_duration} months

1. LinkedIn Advertising (40% - ${int(budget * 0.4):,})
   - Sponsored content and InMail campaigns
   - Target decision makers by job title

2. Content Marketing (30% - ${int(budget * 0.3):,})
   - Technical whitepapers and case studies
   - SEO-optimized blog posts

3. Industry Events & Webinars (20% - ${int(budget * 0.2):,})
   - Virtual and in-person conferences
   - Thought leadership webinars

4. Email Marketing (10% - ${int(budget * 0.1):,})
   - Nurture campaigns to qualified leads
   - Product updates and educational content
"""
    else:  # B2C
        channels = f"""
Recommended Marketing Channels (B2C):
Budget: ${budget:,} over {campaign_duration} months

1. Social Media Advertising (35% - ${int(budget * 0.35):,})
   - Instagram and TikTok for visual products
   - Facebook for broader reach

2. Influencer Marketing (25% - ${int(budget * 0.25):,})
   - Micro and mid-tier influencer partnerships
   - Authentic product reviews and demos

3. Google Ads (20% - ${int(budget * 0.2):,})
   - Search ads for high-intent keywords
   - Shopping ads for e-commerce

4. Email & SMS Marketing (15% - ${int(budget * 0.15):,})
   - Promotional campaigns and cart abandonment
   - Loyalty program communications

5. Content Marketing (5% - ${int(budget * 0.05):,})
   - Blog posts and style guides
   - User-generated content campaigns
"""
    
    return channels


@tool
def calculate_campaign_roi(
    budget: int,
    expected_conversions: int,
    avg_customer_value: int
) -> str:
    """
    Calculate expected ROI for a marketing campaign.
    
    Args:
        budget: Total campaign budget in USD
        expected_conversions: Expected number of conversions
        avg_customer_value: Average customer lifetime value in USD
    
    Returns:
        ROI calculation with key metrics
    """
    
    total_revenue = expected_conversions * avg_customer_value
    roi = ((total_revenue - budget) / budget) * 100
    cost_per_acquisition = budget / expected_conversions if expected_conversions > 0 else 0
    
    calculation = f"""
Campaign ROI Analysis:

Investment:
- Campaign Budget: ${budget:,}
- Cost per Acquisition: ${cost_per_acquisition:,.2f}

Expected Returns:
- Expected Conversions: {expected_conversions:,}
- Average Customer Value: ${avg_customer_value:,}
- Total Revenue: ${total_revenue:,}

ROI Metrics:
- Return on Investment: {roi:.1f}%
- Revenue Multiple: {total_revenue/budget:.2f}x
- {'✅ Profitable campaign' if roi > 0 else '⚠️ Review campaign strategy'}
"""
    
    return calculation


# List all marketing tools
marketing_tools = [
    analyze_target_audience,
    recommend_marketing_channels,
    calculate_campaign_roi
]

print("✅ Defined custom marketing tools:")
for tool_func in marketing_tools:
    print(f"   - {tool_func.__name__}")

### 🤖 Create Tenant-Specific Strands Agents

Now we'll create a function that builds tenant-specific agents with AIP-enabled BedrockModel instances.

**The Key Integration:**
```python
bedrock_model = BedrockModel(
    model_id=aip_arn,  # ← AIP ARN instead of system model ID
    region_name=Region
)

agent = Agent(
    model=bedrock_model,  # ← AIP-enabled model
    tools=marketing_tools
)
```

This pattern automatically routes all agent invocations through the tenant's AIP for isolated tracking.

In [ ]:
def create_tenant_agent(
    tenant_id: str,
    aip_arn: str,
    tools: List
) -> Agent:
    """
    Create a tenant-specific Strands agent with AIP integration.
    
    Args:
        tenant_id: Tenant identifier (tenant_a or tenant_b)
        aip_arn: Application Inference Profile ARN for this tenant
        tools: List of tools available to the agent
    
    Returns:
        Configured Strands Agent instance
    """
    
    # Create BedrockModel with AIP ARN for tenant isolation
    bedrock_model = BedrockModel(
        model_id=aip_arn,  # Use AIP ARN for tenant-specific tracking
        region_name=Region,
        temperature=0.7,
        streaming=True  # Enable streaming for real-time responses
    )
    
    # Define tenant-specific system prompts
    system_prompts = {
        "tenant_a": """
You are a specialized B2B marketing AI assistant for a tech company.

Your expertise:
- B2B SaaS and enterprise software marketing
- Lead generation and demand generation strategies
- ROI-focused campaign planning
- Technical audience engagement

Use the available tools to:
1. Analyze target audiences (decision makers, technical buyers)
2. Recommend B2B marketing channels (LinkedIn, content marketing, events)
3. Calculate campaign ROI with conservative B2B conversion rates

Your responses should be professional, data-driven, and focused on measurable business outcomes.
""",
        "tenant_b": """
You are a specialized B2C retail marketing AI assistant for a fashion company.

Your expertise:
- E-commerce and retail marketing
- Consumer behavior and purchase psychology
- Social media and influencer marketing
- Brand storytelling and visual merchandising

Use the available tools to:
1. Analyze target audiences (consumer demographics and psychographics)
2. Recommend B2C marketing channels (social media, influencers, paid search)
3. Calculate campaign ROI with typical e-commerce conversion rates

Your responses should be creative, trend-aware, and focused on customer engagement and conversion.
"""
    }
    
    # Create agent with tenant-specific configuration
    agent = Agent(
        model=bedrock_model,
        tools=tools,
        system_prompt=system_prompts.get(tenant_id, ""),
        callback_handler=None  # Disable default console output for cleaner notebook
    )
    
    return agent


# Create agents for both tenants
tenant_agents = {}

for tenant_id, aip_arn in tenant_aips.items():
    print(f"\n🤖 Creating Strands agent for {tenant_id}...")
    
    try:
        agent = create_tenant_agent(
            tenant_id=tenant_id,
            aip_arn=aip_arn,
            tools=marketing_tools
        )
        
        tenant_agents[tenant_id] = agent
        
        print(f"✅ Agent created for {tenant_id}")
        print(f"   Model: BedrockModel with AIP")
        print(f"   Tools: {len(marketing_tools)} marketing tools")
        print(f"   AIP ARN: {aip_arn}")
        
    except Exception as e:
        print(f"❌ Error creating agent for {tenant_id}: {str(e)}")

print(f"\n📊 Summary: {len(tenant_agents)} tenant-specific agents ready")

### 📋 Campaign Briefs for Each Tenant

Define the campaign requirements that our agents will work with.

In [ ]:
# Define campaign briefs for each tenant
CAMPAIGN_BRIEFS = {
    "tenant_a": """
Campaign Brief for B2B Tech Company:

Product: New SaaS platform for DevOps automation
Target: CTOs and Engineering Directors at mid-size companies (100-1000 employees)
Goal: Generate 500 qualified leads for product demo
Budget: $50,000 over 3 months
Key Message: "Reduce deployment time by 80% with AI-powered DevOps"

Please create a comprehensive marketing campaign that includes:
1. Target audience analysis
2. Recommended marketing channels with budget allocation
3. ROI projections (assume 5% conversion rate, $25,000 avg customer value)
4. Campaign theme and messaging strategy
""",
    "tenant_b": """
Campaign Brief for B2C Retail Company:

Product: Summer fashion collection for millennials
Target: Fashion-conscious millennials aged 25-35 with disposable income
Goal: Increase online sales by 25% during summer season
Budget: $75,000 over 2 months
Key Message: "Sustainable fashion that doesn't compromise on style"

Please create a comprehensive marketing campaign that includes:
1. Target audience analysis
2. Recommended marketing channels with budget allocation
3. ROI projections (assume 3% conversion rate, $150 avg order value)
4. Campaign theme and messaging strategy
"""
}

print("📋 Campaign briefs defined for both tenants")
print(f"   - tenant_a: B2B Tech (DevOps SaaS)")
print(f"   - tenant_b: B2C Retail (Fashion)")

### 🎯 Generate Marketing Campaigns with Strands Agents

Now let's invoke our tenant-specific agents to generate marketing campaigns.

**What happens:**
1. Agent receives the campaign brief
2. Agent analyzes the request using its reasoning capabilities
3. Agent automatically decides which tools to use (analyze_target_audience, recommend_channels, etc.)
4. Agent orchestrates tool calls in the right order
5. Agent synthesizes results into a comprehensive campaign
6. All invocations tracked via the tenant's AIP in CloudWatch

**Strands Magic:**
- No manual tool logic needed
- No manual token tracking
- Automatic agent loop handles complexity
- Built-in metrics in `AgentResult`

In [ ]:
# Generate campaigns using Strands agents
campaign_results = {}

for tenant_id in TENANT_CONFIGS.keys():
    if tenant_id in tenant_agents:
        print(f"\n{'='*80}")
        print(f"🎯 GENERATING CAMPAIGN FOR {tenant_id.upper()}")
        print(f"{'='*80}\n")
        
        try:
            # Get the agent for this tenant
            agent = tenant_agents[tenant_id]
            
            # Invoke the agent with the campaign brief
            print(f"🤖 Agent is working...\n")
            start_time = datetime.now()
            
            result = agent(CAMPAIGN_BRIEFS[tenant_id])
            
            end_time = datetime.now()
            latency = (end_time - start_time).total_seconds()
            
            # Store result
            campaign_results[tenant_id] = {
                'result': result,
                'latency': latency,
                'timestamp': datetime.now().isoformat()
            }
            
            print(f"\n✅ Campaign generated for {tenant_id}")
            print(f"   Latency: {latency:.2f}s")
            print(f"   Agent cycles: {result.metrics.cycle_count}")
            
        except Exception as e:
            print(f"\n❌ Error generating campaign for {tenant_id}: {str(e)}")
            campaign_results[tenant_id] = {
                'error': str(e),
                'timestamp': datetime.now().isoformat()
            }

print(f"\n{'='*80}")
print(f"📊 Campaign generation complete for {len(campaign_results)} tenants")
print(f"{'='*80}")

## Section 3: Review Results and Metrics

Let's examine the generated campaigns and the built-in metrics from Strands agents.

### 📝 Display Generated Marketing Campaigns

Review the AI-generated campaigns for each tenant.

In [ ]:
# Display generated campaigns
for tenant_id, campaign_data in campaign_results.items():
    print(f"\n{'='*80}")
    print(f"📄 CAMPAIGN FOR {tenant_id.upper()}")
    print(f"{'='*80}\n")
    
    if 'error' in campaign_data:
        print(f"❌ Error: {campaign_data['error']}")
    else:
        result = campaign_data['result']
        print(result.message['content'][0]['text'])
        
        print(f"\n{'─'*80}")
        print(f"⏱️ Generation time: {campaign_data['latency']:.2f}s")
        print(f"🔄 Agent cycles: {result.metrics.cycle_count}")

### 📊 Display Agent Metrics and Tool Usage

Strands agents provide comprehensive built-in metrics through `AgentResult.metrics`.

**What you get automatically:**
- Token usage (input, output, total)
- Latency metrics
- Tool usage statistics (calls, success rate, execution time)
- Agent cycles and traces

In [ ]:
# Display detailed agent metrics
for tenant_id, campaign_data in campaign_results.items():
    if 'error' not in campaign_data:
        result = campaign_data['result']
        
        print(f"\n{'='*80}")
        print(f"📊 AGENT METRICS FOR {tenant_id.upper()}")
        print(f"{'='*80}\n")
        
        # Token usage
        usage = result.metrics.accumulated_usage
        print("🔢 Token Usage:")
        print(f"   Input Tokens:  {usage.get('inputTokens', 0):,}")
        print(f"   Output Tokens: {usage.get('outputTokens', 0):,}")
        print(f"   Total Tokens:  {usage.get('totalTokens', 0):,}")
        
        # Latency
        metrics = result.metrics.accumulated_metrics
        cycle_durations = result.metrics.cycle_durations
        avg_cycle_time = sum(cycle_durations) / len(cycle_durations) if cycle_durations else 0
        
        print(f"\n⏱️ Performance:")
        print(f"   Total Latency: {metrics.get('latencyMs', 0):,.0f}ms")
        print(f"   Agent Cycles:  {result.metrics.cycle_count}")
        print(f"   Avg Cycle Time: {avg_cycle_time:.2f}s")
        
        # Tool usage
        print(f"\n🛠️ Tool Usage:")
        if result.metrics.tool_metrics:
            for tool_name, tool_data in result.metrics.tool_metrics.items():
                print(f"\n   {tool_name}:")
                print(f"      Calls: {tool_data.call_count}")
                print(f"      Success: {tool_data.success_count}")
                print(f"      Errors: {tool_data.error_count}")
                success_rate = (tool_data.success_count / tool_data.call_count * 100) if tool_data.call_count > 0 else 0
                print(f"      Success Rate: {success_rate:.0f}%")
                avg_time_ms = (tool_data.total_time / tool_data.call_count * 1000) if tool_data.call_count > 0 else 0
                print(f"      Avg Execution: {avg_time_ms:.2f}ms")
        else:
            print("   No tools were used by the agent")
        
        print(f"\n{'─'*80}")

### 🟢 AFTER Check: CloudWatch Metrics Show Multi-Tenant Isolation

Now let's check CloudWatch metrics after agent invocations to see per-tenant tracking in action.

**Expected results:**
- Separate metrics for tenant_a and tenant_b
- Invocations, input tokens, output tokens tracked per tenant
- Visual proof of multi-tenant isolation

⏳ **Note:** We wait 90 seconds for CloudWatch metrics to propagate.

In [ ]:
# Wait for CloudWatch metrics to propagate
print("⏳ Waiting 60 seconds for CloudWatch metrics to propagate...")
time.sleep(60)
print("✅ Wait complete - proceeding with metrics monitoring\n")

# Check CloudWatch metrics AFTER agent invocations
print("🟢 AFTER CHECK: Querying CloudWatch for post-campaign metrics...")
print("="*80)
print("Expected: Per-tenant metrics showing isolated usage!")
print("="*80 + "\n")

after_metrics = monitor_aip_usage(tenant_aips, Region)

print("\n" + "="*80)
print("✅ AFTER check complete!")
print("💡 Compare BEFORE vs AFTER to see proof of multi-tenant isolation")
print("="*80)

### 🔍 Comparison: Boto3 (Lab 03) vs Strands (Lab 04)

Let's compare the two approaches side-by-side:


In [ ]:
print("""
╔═══════════════════════════════════════════════════════════════════════════════╗
║                    BOTO3 (Lab 03) vs STRANDS (Lab 04)                         ║
╚═══════════════════════════════════════════════════════════════════════════════╝

┌─────────────────────┬──────────────────────────┬──────────────────────────────┐
│ Aspect              │ Lab 03: Boto3            │ Lab 04: Strands Agents       │
├─────────────────────┼──────────────────────────┼──────────────────────────────┤
│ API Style           │ Direct API calls         │ Agent framework              │
│                     │ invoke_model()           │ agent(prompt)                │
├─────────────────────┼──────────────────────────┼──────────────────────────────┤
│ AIP Integration     │ modelId=aip_arn          │ BedrockModel(model_id=arn)   │
│                     │ Manual parameter passing │ Model abstraction            │
├─────────────────────┼──────────────────────────┼──────────────────────────────┤
│ Tool Definition     │ Manual function calls    │ @tool decorator              │
│                     │ Hardcoded logic          │ Automatic discovery          │
├─────────────────────┼──────────────────────────┼──────────────────────────────┤
│ Tool Orchestration  │ Manual control flow      │ Automatic agent loop         │
│                     │ Developer decides        │ LLM decides                  │
├─────────────────────┼──────────────────────────┼──────────────────────────────┤
│ Token Tracking      │ Manual extraction        │ result.metrics.usage         │
│                     │ Parse response body      │ Automatic capture            │
├─────────────────────┼──────────────────────────┼──────────────────────────────┤
│ Metrics             │ CloudWatch only          │ Agent metrics + CloudWatch   │
│                     │ Query separately         │ Built-in observability       │
├─────────────────────┼──────────────────────────┼──────────────────────────────┤
│ Streaming           │ Manual parsing           │ Built-in callback handlers   │
│                     │ Handle chunks manually   │ Automatic streaming          │
├─────────────────────┼──────────────────────────┼──────────────────────────────┤
│ Error Handling      │ Try/catch per call       │ Built-in retry logic         │
│                     │ Manual retry logic       │ Automatic error handling     │
├─────────────────────┼──────────────────────────┼──────────────────────────────┤
│ Multi-step Tasks    │ Manual iteration         │ Automatic agent cycles       │
│                     │ Developer orchestrates   │ Agent orchestrates           │
├─────────────────────┼──────────────────────────┼──────────────────────────────┤
│ Code Complexity     │ ~100 lines per task      │ ~20 lines per task           │
│                     │ More boilerplate         │ Less boilerplate             │
├─────────────────────┼──────────────────────────┼──────────────────────────────┤
│ Development Speed   │ Slower                   │ Faster                       │
│                     │ More manual work         │ Framework handles details    │
├─────────────────────┼──────────────────────────┼──────────────────────────────┤
│ Best For            │ Simple invocations       │ Complex agent applications   │
│                     │ Full control needed      │ Multi-step reasoning         │
│                     │ Custom integrations      │ Tool-based workflows         │
└─────────────────────┴──────────────────────────┴──────────────────────────────┘

💡 KEY INSIGHT: Both approaches work with AIPs for tenant isolation!
   - Boto3: More control, more code
   - Strands: Less code, built-in features, automatic orchestration

🎯 WHEN TO USE EACH:
   - Use Boto3 when you need fine-grained control over every API call
   - Use Strands when building agent applications with tools and reasoning
""")

## Lab Summary

🎉 **Congratulations!** You've successfully integrated Application Inference Profiles with Strands Agents for a multi-tenant Marketing AI platform.

### What You Accomplished:

1. **✅ Reused or created Application Inference Profiles** from Lab 03
2. **✅ Integrated AIPs with Strands BedrockModel** for tenant isolation
3. **✅ Defined custom marketing tools** using `@tool` decorator
4. **✅ Created tenant-specific agents** with AIP-enabled models
5. **✅ Generated AI campaigns** with automatic tool orchestration
6. **✅ Accessed built-in agent metrics** (tokens, latency, tool usage)
7. **✅ Verified CloudWatch isolation** with before/after checks
8. **✅ Compared Boto3 vs Strands approaches**

### Key Takeaways:

#### **🎯 AIP Integration Pattern:**
```python
# The pattern that enables tenant isolation
bedrock_model = BedrockModel(model_id=aip_arn)
agent = Agent(model=bedrock_model, tools=[tools])
```

#### **🛠️ Tool Definition:**
- Use `@tool` decorator for automatic discovery
- Type hints become tool schema
- Docstrings guide the LLM

#### **🤖 Agent Loop:**
- Automatic reasoning and tool selection
- No manual orchestration needed
- Multi-step tasks handled automatically

#### **📊 Built-in Observability:**
- `result.metrics.accumulated_usage` → Token counts (inputTokens, outputTokens, totalTokens)
- `result.metrics.tool_metrics` → Per-tool statistics (call_count, success_count, total_time)
- `result.metrics.cycle_count` → Number of agent iterations
- `result.metrics.cycle_durations` → Duration of each agent cycle

#### **🔍 Multi-Tenant Isolation:**
- Same underlying model (Claude Sonnet 4)
- Separate tracking per tenant via AIP
- Per-tenant CloudWatch metrics
- Accurate billing and SLA monitoring

### Boto3 vs Strands Decision Matrix:

| Use Boto3 When... | Use Strands When... |
|-------------------|---------------------|
| Need fine-grained API control | Building agent applications |
| Simple request/response pattern | Multi-step reasoning needed |
| Custom integration requirements | Want automatic tool orchestration |
| Minimal dependencies preferred | Rich observability needed |
| Direct model invocation | Complex workflows with tools |

### Next Steps:

Continue your AIP journey with other frameworks:
- **Lab 05**: AgentCore Runtime - Deploy Strands agents to production with full observability
- **Lab 06 (Optional)**: LiteLLM - AI gateway patterns with AIP
- **Lab 07 (Optional)**: LangChain - Chain-based applications with tenant awareness
- **Lab 08 (Optional)**: LangGraph - Graph workflows for complex multi-step processes

### 💡 Pro Tips:

1. **Tool Design**: Keep tools focused and single-purpose for better agent decision-making
2. **System Prompts**: Customize per tenant for domain-specific behavior
3. **Metrics**: Use `result.metrics` for immediate feedback during development
4. **CloudWatch**: Use for production monitoring and billing
5. **Caching**: Enable prompt/tool caching in BedrockModel for cost optimization

**Ready for production deployment?** → [Continue to Lab 05: AgentCore Runtime](lab-05-agentcore-runtime-deployment.ipynb)

**Ready to explore gateway patterns?** → [Skip to Lab 06: LiteLLM (Optional)](lab-06-aip-litellm-optional.ipynb)